In [9]:
! pip install selenium

  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-2.0.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------ --------------------------------- 1.6/9.6 MB 11.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 13.2 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.6 MB 12.2 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.6 MB 11.3 MB/s eta 0:00:01
   --------

##  Ambil List Kode dan Nama Kecamatan

Tahapan ini berfungsi untuk **mengambil dan memetakan daftar kode serta nama kecamatan** di Kabupaten Semarang, yang kemudian digunakan dalam dua fungsi utama sistem:

1. **Sebagai dasar penyusunan URL dinamis untuk proses scraping**, dan  
2. **Sebagai sumber data untuk dropdown pemilihan kecamatan pada tahap antarmuka (deployment)**

---
Setiap halaman daftar sekolah pada situs resmi **Kemendikdasmen** mengikuti pola URL yang tetap:
https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3

Bagian `{kode_kecamatan}` menunjukkan **identitas numerik unik** untuk setiap kecamatan.  
Contoh:
- Kecamatan *Ambarawa* memiliki kode `032210`,  

Untuk mendapatkan kode ini secara sistematis, sistem membaca file JSON lokal (misalnya `kecamatan_kab_semarang.json`) yang berisi pasangan:

```json
{
  "Kab. Semarang": {
    "kode": "032200",
    "kecamatan": {
      "Ambarawa": "032210",
      "Tengaran": "032202",
      "Susukan": "032203",
      ....
    }
  }
}




In [ ]:
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


def setup_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    # options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--log-level=3")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.page_load_strategy = "eager"
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver


def get_table_rows(driver, url):
    """Helper untuk ambil semua baris dari tabel referensi"""
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr"))
    )
    try:
        select = Select(driver.find_element(By.NAME, "table1_length"))
        select.select_by_value("100")
    except:
        pass
    return driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")


def get_kecamatan_jateng_by_kode():
    """
    Ambil daftar kabupaten/kota di Jawa Tengah (030000),
    input kode kabupaten/kota (misal 032200),
    hasil disimpan ke data/kecamatan_<nama_kab>.json
    dalam format JSON hierarkis.
    """
    base_url = "https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/"
    driver = setup_driver(headless=True)
    kecamatan_list = []

    try:
        # 1️⃣ Ambil daftar kabupaten/kota di Provinsi Jawa Tengah
        print("Mengambil daftar kabupaten/kota di Provinsi Jawa Tengah...")
        kab_url = base_url + "030000/1"
        rows_kab = get_table_rows(driver, kab_url)

        kabupaten_list = []
        for r in rows_kab:
            try:
                nama = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                kode = href.split("/")[-2]
                kabupaten_list.append({"nama": nama, "kode": kode})
            except:
                continue

        print("\nDaftar Kabupaten/Kota di Jawa Tengah:")
        print("=" * 60)
        for k in kabupaten_list:
            display(f"{k['kode']} - {k['nama']}")
        print("=" * 60)

        kode_input = input("\nMasukkan KODE kabupaten/kota yang ingin discrap (contoh: 032200): ").strip()

        kab = next((k for k in kabupaten_list if k["kode"] == kode_input), None)
        if not kab:
            print("Kode tidak ditemukan dalam daftar.")
            driver.quit()
            return []

        print(f"\nMengambil daftar kecamatan di {kab['nama']}...")

        # 2️⃣ Ambil daftar kecamatan dari kabupaten/kota terpilih
        kec_url = base_url + f"{kab['kode']}/2"
        rows_kec = get_table_rows(driver, kec_url)

        for r in rows_kec:
            try:
                nama = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                kode = href.split("/")[-2]
                kecamatan_list.append({"nama": nama, "kode": kode, "url": href})
            except:
                continue

        print(f"Berhasil ambil {len(kecamatan_list)} kecamatan dari {kab['nama']}")

        # 3️⃣ Simpan hasil ke list_kecamatan/
        os.makedirs("list_kecamatan", exist_ok=True)
        safe_name = kab["nama"].replace(" ", "_").replace(".", "").lower()
        save_path = os.path.join("list_kecamatan", f"kecamatan_{safe_name}.json")

        # === Format sesuai contoh Prof ===
        json_data = {
            kab["nama"]: {
                "kode": kab["kode"],
                "kecamatan": {k["nama"]: k["kode"] for k in kecamatan_list}
            }
        }

        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)

        print(f"Data disimpan ke: {save_path}")

    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

    driver.quit()
    return kecamatan_list


# ==========================
# MAIN
# ==========================
if __name__ == "__main__":
    kecamatan_data = get_kecamatan_jateng_by_kode()

Mengambil daftar kabupaten/kota di Provinsi Jawa Tengah...

Daftar Kabupaten/Kota di Jawa Tengah:


'030100 - Kab. Cilacap'

'030200 - Kab. Banyumas'

'030300 - Kab. Purbalingga'

'030400 - Kab. Banjarnegara'

...

'036200 - Kota Salatiga'

'036300 - Kota Semarang'

'036400 - Kota Pekalongan'

'036500 - Kota Tegal'


Mengambil daftar kecamatan di Kab. Semarang...
Berhasil ambil 19 kecamatan dari Kab. Semarang
Data disimpan ke: data\kecamatan_kab_semarang.json


# GRADIO

In [2]:
! pip install undetected-chromedriver beautifulsoup4 requests gradio

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47215 sha256=07c194f4370a3b0397cf810efb78f1afe7a663f4f05e3e0c8766294aff2a59cf
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\cf\a1\db\e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected-chromedriver


  DEPRECATION: Building 'undetected-chromedriver' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'undetected-chromedriver'. Discussion can be found at https://github.com/pypa/pip/issues/6334


## ⚙️ Versi 1 — Full Selenium WebDriver Scraper

Versi pertama dari sistem scraper ini menggunakan **Selenium WebDriver sepenuhnya** untuk melakukan otomatisasi pengambilan data sekolah dari situs  
[`referensi.data.kemendikdasmen.go.id`](https://referensi.data.kemendikdasmen.go.id).  
Seluruh proses — mulai dari membuka halaman utama, navigasi antar tab, hingga redirect ke halaman profil sekolah — dilakukan secara **langsung di browser Chrome** melalui kontrol Selenium.

Pendekatan ini **tidak menggunakan `requests` maupun `BeautifulSoup`**, melainkan sepenuhnya mengandalkan simulasi interaksi pengguna (click, scroll, open tab, dsb).  
Metode ini dirancang agar seluruh elemen web yang dimuat secara dinamis (JavaScript-rendered) dapat terbaca sepenuhnya sebelum data diekstraksi.

---

### 🔹 Metode & Arsitektur

- **Selenium WebDriver (Chrome)** digunakan untuk membuka halaman, menavigasi antar tab, dan mengeksekusi interaksi pengguna secara otomatis.
- Semua proses — mulai dari pengambilan daftar sekolah hingga data detail — dilakukan **sinkron dan sekuensial** tanpa multi-threading.
- **Navigasi antar domain** (dari `referensi.data.kemendikdasmen.go.id` ke `sekolah.data.kemdikbud.go.id`) dilakukan dengan membuka **tab baru secara langsung di browser**.
- **Ekstraksi data** dilakukan menggunakan kombinasi `CSS Selector` dan `XPath`.
- **Gradio UI** digunakan untuk menyediakan antarmuka pengguna dalam memilih kecamatan dan kolom data yang ingin diambil.

---

### 🔹 Alur Proses

1. **Membuka Halaman Referensi Sekolah**  
   Berdasarkan *kode kecamatan* dari file JSON, Selenium membuka halaman referensi untuk jenjang SD (`value=5`) dan MI (`value=9`).

2. **Mengambil Daftar Sekolah (Tabel Utama)**  
   Selenium membaca tabel `table#table1` untuk mengambil data dasar:  
   - Nama Sekolah  
   - NPSN  
   - Status  
   - Kelurahan  

3. **Redirect ke Profil Sekolah (Tab Identitas)**  
   Setiap baris sekolah memiliki tautan menuju halaman profil di domain  
   `https://sekolah.data.kemdikbud.go.id`.  
   Selenium membuka link ini di tab baru menggunakan perintah:
   ```python
   driver.execute_script("window.open(arguments[0]);", link_kemdikbud)
   driver.switch_to.window(driver.window_handles[-1])
   Dari halaman tersebut, Selenium mengekstrak:
   -  Alamat sekolah
   -  Kepala sekolah
   -  Jumlah siswa laki-laki
   -  Jumlah siswa perempuan
4. **Navigasi ke Tab Kontak (Tab 4)**

   Selenium kemudian berpindah ke tab kontak di halaman sekolah untuk mengambil data tambahan:
   - Nomor telepon
   - Email
   - Website sekolah

5. **Kembali ke Halaman Utama**
   
   Setelah data diambil, Selenium menutup tab profil dan kembali ke tab utama untuk melanjutkan ke sekolah berikutnya.
6. **Penyimpanan Data ke CSV**
   
   Semua hasil ekstraksi disusun ke dalam list Python dan disimpan sebagai file .csv
di folder output/, sesuai dengan nama kecamatan yang dipilih pengguna.

---

### 🔹 Ciri Teknis & Karakteristik
| Aspek                   | Keterangan                                                                |
| ----------------------- | ------------------------------------------------------------------------- |
| **Metode Scraping**     | Full Selenium Automation (tanpa `requests` / `BeautifulSoup`)             |
| **Parallel Processing** | Tidak (sinkron, satu per satu)                                            |
| **Redirect Handling**   | `window.open()` + `switch_to.window()`                                    |
| **Teknik Ekstraksi**    | Kombinasi CSS Selector & XPath                                            |
| **Driver**              | Chrome WebDriver (headless mode)                                          |
| **Output**              | File CSV berisi daftar lengkap sekolah SD/MI per kecamatan                |
| **Keunggulan**          | Akurat, stabil, dan mampu membaca halaman dinamis berbasis JavaScript     |
| **Kelemahan**           | Lambat karena browser memuat seluruh halaman secara penuh setiap redirect |
| **Estimasi Waktu**      | ±5–8 Menit per Kecamatan                                                  |

### 🔹 Ringkasan

Versi 1 berfungsi sebagai baseline sistem scraping penuh berbasis Selenium, di mana seluruh proses dikontrol langsung oleh browser.
Selenium meniru interaksi manual pengguna untuk memastikan seluruh konten termuat sebelum pengambilan data dilakukan.

Pendekatan ini paling akurat, namun juga paling berat secara sumber daya dan waktu, sehingga menjadi dasar pengembangan menuju versi 2 yang lebih cepat dan efisien.



In [2]:
import os
import csv
import json
import time
import gradio as gr
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from time import sleep


# =====================================================
#  SETUP SELENIUM CHROME DRIVER
# =====================================================
def setup_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--log-level=3")
    options.page_load_strategy = "eager"
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver


def get_table_rows(driver, url):
    """Helper untuk ambil semua baris dari tabel referensi"""
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr"))
    )
    try:
        select = Select(driver.find_element(By.NAME, "table1_length"))
        select.select_by_value("100")
    except:
        pass
    return driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")

# =====================================================
#  AMBIL KODE KECAMATAN DARI JSON
# =====================================================
def get_kode_kecamatan_from_json(nama_kecamatan, json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    try:
        # Buka file JSON
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        if not data:
            print("File JSON kosong.")
            return None

        nama_wilayah = next(iter(data))
        kabupaten_data = data[nama_wilayah].get("kecamatan", {})

        # Cari kecamatan
        for nama, kode in kabupaten_data.items():
            if nama.lower() == nama_kecamatan.lower():
                return kode

        print(f"Kecamatan '{nama_kecamatan}' tidak ditemukan di {nama_wilayah}.")
        return None

    except FileNotFoundError:
        print(f"File JSON '{json_path}' tidak ditemukan.")
        return None

    except Exception as e:
        print(f"Error saat membaca JSON: {e}")
        return None

# =====================================================
#  DETAIL SEKOLAH (ALAMAT, KEPSEK, SISWA)
# =====================================================
def get_detail(driver):
    alamat, kepsek, siswa_laki, siswa_perempuan = "-", "-", "-", "-"
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='tab-1']"))
        )
        rows_identitas = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")
        for rp in rows_identitas:
            tds = rp.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue
            link_elem = tds[3].find_elements(By.TAG_NAME, "a")
            if not link_elem:
                continue
            link_kemdikbud = link_elem[0].get_attribute("href")

            # Buka tab baru
            driver.execute_script("window.open(arguments[0]);", link_kemdikbud)
            driver.switch_to.window(driver.window_handles[-1])

            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h4.page-header"))
                )

                # Alamat
                try:
                    alamat_elem = driver.find_element(By.CSS_SELECTOR, "font.small")
                    alamat_text = alamat_elem.text.replace("(master referensi)", "").strip()
                    alamat = alamat_text if alamat_text else "-"
                except:
                    alamat = "-"

                # Kepala Sekolah
                try:
                    kepsek_elem = driver.find_element(By.XPATH, "//li[contains(., 'Kepala Sekolah')]")
                    kepsek = kepsek_elem.text.split(":", 1)[-1].strip()
                except:
                    kepsek = "-"

                # Jumlah Siswa Laki-laki
                try:
                    siswa_laki_elem = driver.find_element( By.XPATH, "//text()[contains(., 'Siswa Laki-laki')]/following::font[1]")
                    siswa_laki = siswa_laki_elem.text.strip()
                except:
                    siswa_laki = "-"

                # Jumlah Siswa Perempuan
                try:
                    siswa_perempuan_elem = driver.find_element(By.XPATH, "//text()[contains(., 'Siswa Perempuan')]/following::font[1]")
                    siswa_perempuan = siswa_perempuan_elem.text.strip()
                except:
                    siswa_perempuan = "-"

            except Exception as e:
                print(f"Gagal ambil data detail: {e}")

            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            break
    except Exception as e:
        print(f"Gagal proses tab identitas: {e}")

    return alamat, kepsek, siswa_laki, siswa_perempuan


# =====================================================
#  KONTAK SEKOLAH
# =====================================================
def get_kontak(driver):
    telepon, email, website = "-", "-", "-"
    try:
        driver.find_element(By.CSS_SELECTOR, "label[for='tab-4']").click()
        sleep(0.5)
        rows_kontak = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")
        for row in rows_kontak:
            tds = row.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue
            label = tds[1].text.strip().lower()
            value = tds[3].text.strip() or "-"
            if "telepon" in label:
                telepon = value if len(value) >= 5 else "-"
            elif "email" in label:
                email = value
            elif "website" in label:
                val = value.lower()
                website = "-" if val in ["http://-", "https://-"] else val
    except:
        pass
    return telepon, email, website


# =====================================================
#  AMBIL DATA SEKOLAH (SD & MI)
# =====================================================
def get_sd_mi_schools(kode_kecamatan, nama_kecamatan, selected_fields, progress=gr.Progress()):
    driver = setup_driver(headless=True)
    sekolah_list = []

    need_detail = any(f in selected_fields for f in ["Alamat", "Kepala Sekolah", "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"])
    need_kontak = any(f in selected_fields for f in ["Telepon", "Email", "Website"])

    for jenjang, value in [("SD", "5"), ("MI", "9")]:
        url = f"https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3/all/{value}/all"
        driver.get(url)
        sleep(0.5)
        print(f"Mengambil data {jenjang} di {nama_kecamatan}...")

        all_rows = []
        try:
            get_table_rows(driver, url)
            while True:
                rows = driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")
                all_rows.extend(rows)
                next_btn = driver.find_element(By.CSS_SELECTOR, "#table1_next")
                if "disabled" in next_btn.get_attribute("class"):
                    break
                driver.execute_script("arguments[0].click();", next_btn)
                sleep(1.2)
        except:
            continue

        total = len(all_rows)
        processed = 0

        for r in all_rows:
            data = {}
            try:
                if "Nama Sekolah" in selected_fields:
                    data["Nama Sekolah"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
                if "NPSN" in selected_fields:
                    data["NPSN"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                if "Status" in selected_fields:
                    data["Status"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text.strip()
                if "Kelurahan" in selected_fields:
                    data["Kelurahan"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text.strip()

                if need_detail or need_kontak:
                    href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                    driver.execute_script("window.open(arguments[0]);", href)
                    driver.switch_to.window(driver.window_handles[-1])

                    if need_detail:
                        alamat, kepsek, siswa_laki, siswa_perempuan = get_detail(driver)
                        if "Alamat" in selected_fields: data["Alamat"] = alamat
                        if "Kepala Sekolah" in selected_fields: data["Kepala Sekolah"] = kepsek
                        if "Jumlah Siswa Laki-laki" in selected_fields: data["Jumlah Siswa Laki-laki"] = siswa_laki
                        if "Jumlah Siswa Perempuan" in selected_fields: data["Jumlah Siswa Perempuan"] = siswa_perempuan

                    if need_kontak:
                        telepon, email, website = get_kontak(driver)
                        if "Telepon" in selected_fields: data["Telepon"] = telepon
                        if "Email" in selected_fields: data["Email"] = email
                        if "Website" in selected_fields: data["Website"] = website

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                sekolah_list.append(data)
            except:
                pass
            processed += 1
            if total > 0:
                progress(processed / total)

    driver.quit()

    # ===== SORTING berdasarkan prioritas =====
    sort_priority = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]
    sort_key = next((f for f in sort_priority if f in selected_fields), None)
    if sort_key:
        sekolah_list.sort(key=lambda x: x.get(sort_key, "").lower())
    return sekolah_list


# =====================================================
#  SIMPAN CSV
# =====================================================
def save_school_list_by_kecamatan(nama_kecamatan, selected_fields):
    kode_kecamatan = get_kode_kecamatan_from_json(nama_kecamatan)
    sekolah_list = get_sd_mi_schools(kode_kecamatan, nama_kecamatan, selected_fields)
    if not sekolah_list:
        return f" Tidak ada data untuk '{nama_kecamatan}'.", None

    folder = "output"
    os.makedirs(folder, exist_ok=True)
    filename = f"list_sd_mi_{nama_kecamatan.lower().replace(' ', '_')}.csv"
    path = os.path.join(folder, filename)

    full_order = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]
    ordered_fields = [col for col in full_order if col in selected_fields]

    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=ordered_fields)
        writer.writeheader()
        writer.writerows(sekolah_list)
    return f"{len(sekolah_list)} sekolah disimpan ke '{path}'", path


# =====================================================
#  GRADIO UI
# =====================================================
def run_scraper(nama_kecamatan, selected_fields):
    if nama_kecamatan in ["", "-- Pilih Kecamatan --"]:
        yield "Silakan pilih kecamatan terlebih dahulu.", gr.update(visible=False)
        return
    if not selected_fields:
        yield "Pilih minimal satu kolom sebelum memulai scraping.", gr.update(visible=False)
        return

    start_time = time.time()
    status, file_path = save_school_list_by_kecamatan(nama_kecamatan, selected_fields)
    durasi = time.time() - start_time
    menit, detik = divmod(int(durasi), 60)
    waktu_str = f"\nWaktu: {menit} menit {detik} detik" if menit else f"\nWaktu: {detik} detik"

    if file_path:
        yield f"{status}{waktu_str}", gr.update(value=file_path, visible=True)
    else:
        yield f"{status}{waktu_str}", gr.update(visible=False)


def create_gradio_ui(json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    nama_wilayah = next(iter(data))
    kecamatan_list = sorted(data[nama_wilayah].get("kecamatan", {}).keys())

    fields = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]

    with gr.Blocks(title="Scraper SD/MI Kabupaten Semarang") as demo:
        gr.Markdown("## Scraper SD/MI Kabupaten Semarang v1.0 Selenium WebDriver Full")
        kecamatan = gr.Dropdown(
            label="Pilih Kecamatan",
            choices=["-- Pilih Kecamatan --"] + kecamatan_list,
            value="-- Pilih Kecamatan --",
            interactive=True
        )
        kolom = gr.CheckboxGroup(label="Pilih Kolom CSV", choices=fields)
        tombol = gr.Button("Mulai Scrape")
        status = gr.Textbox(label="Status", lines=3)
        file = gr.File(label="File CSV", interactive=False, visible=False)
        tombol.click(fn=run_scraper, inputs=[kecamatan, kolom], outputs=[status, file])
    return demo


ui = create_gradio_ui()
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Mengambil data SD di Ambarawa...
Mengambil data MI di Ambarawa...
Mengambil data MI di Ambarawa...


## ⚙️ Versi 2 — Hybrid (Selenium + Requests) Scraper

Versi kedua merupakan **penyempurnaan besar** dari sistem scraping sebelumnya, dengan fokus pada **kecepatan, stabilitas koneksi, dan efisiensi sumber daya**.  
Pendekatan ini menggabungkan **Selenium**, **Requests**, dan **ThreadPoolExecutor** secara optimal untuk memaksimalkan performa tanpa mengorbankan akurasi data.

Selenium kini digunakan **hanya untuk tahap awal** (navigasi dan pembacaan tabel dinamis), sementara seluruh proses pengambilan detail sekolah dijalankan menggunakan **HTTP session pooling (keep-alive)** dan **multi-threaded requests**.  

---

### 🔹 Metode & Arsitektur

- **Undetected ChromeDriver (uc)** digunakan untuk mengakses halaman referensi utama dan memuat tabel sekolah yang bersifat dinamis.
- **Requests.Session** digunakan untuk membuat koneksi HTTP cepat dengan sistem **keep-alive pooling** (hingga 50 koneksi simultan).
- **BeautifulSoup** bertugas melakukan parsing cepat terhadap HTML hasil request dari setiap sekolah.
- **ThreadPoolExecutor (25 workers)** mempercepat proses pengambilan detail sekolah (Tab-1 & Tab-4) secara paralel.
- **Gradio UI** dipertahankan untuk memudahkan pemilihan kecamatan, kolom data, dan pemantauan progress scraping.

---

### 🔹 Alur Proses

1. **Inisialisasi Chrome Driver (Undetected)**  
   Selenium digunakan hanya untuk memuat tabel sekolah dari halaman referensi berikut:
    https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3/all/{jenjang}/all
    untuk jenjang SD (5) dan MI (9).
2. **Ekstraksi Daftar Sekolah (via Selenium)**  
Selenium membaca tabel `#table1` untuk memperoleh data dasar:
- Nama Sekolah  
- NPSN  
- Status  
- Kelurahan  
Serta menyimpan setiap **tautan detail sekolah** untuk diproses paralel.

3. **Parallel Fetch Detail (Tab-1 + Tab-4)**  
Dengan `ThreadPoolExecutor(max_workers=25)`, sistem secara paralel:
- Memanggil **Tab Identitas (Tab-1)** → mengambil *Alamat*, *Kepala Sekolah*, *Jumlah Siswa L/P*  
- Memanggil **Tab Kontak (Tab-4)** → mengambil *Telepon*, *Email*, *Website*

4. **Kompilasi & Penyimpanan Data**  
Hasil seluruh threads digabung dan disimpan dalam file `.csv` di folder `output/`, dengan urutan kolom sesuai pilihan pengguna.

---

### 🔹 Ciri Teknis & Karakteristik

| Aspek                     | Keterangan                                                                 |
| -------------------------- | -------------------------------------------------------------------------- |
| **Metode Scraping**       | Hybrid++: Selenium (UC) + Requests (Session Pool) + BeautifulSoup + ThreadPoolExecutor |
| **Parallel Processing**   | Ya — hingga 25 thread sekaligus                                            |
| **Optimasi Koneksi**      | HTTP Keep-Alive dengan Pool 50 koneksi aktif                               |
| **Driver**                | Undetected ChromeDriver (`uc.Chrome`)                                      |
| **Teknik Ekstraksi**      | Selenium untuk tabel, BeautifulSoup untuk parsing detail                   |
| **Output**                | File CSV berisi daftar lengkap SD/MI per kecamatan                         |
| **Keunggulan**            | Kecepatan tinggi, koneksi stabil, mampu menangani domain redirect dinamis  |
| **Kelemahan**             | Lebih kompleks; konsumsi bandwidth lebih besar karena multi-thread request |
| **Estimasi Waktu**        | ±35 detik – 1.5 menit per Kecamatan                                          |

---

### 🔹 Ringkasan

Versi 2 menghadirkan sistem scraping **paling efisien dan stabil** dalam proyek ini.  
Dengan pemisahan yang jelas antara proses dinamis (Selenium) dan proses statis (Requests), versi ini memberikan peningkatan performa hingga **5–6× lebih cepat dari versi awal**, sekaligus mengurangi potensi error akibat perubahan domain atau keterlambatan jaringan.

Sistem ini siap digunakan untuk **deployment di server backend**, **otomatisasi pengumpulan data pendidikan**, dan **pengolahan data berskala besar antar kecamatan**.


In [ ]:
import os, csv, json, time, gradio as gr, requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


# =====================================================
#  SETUP DRIVER UNTUK LOKAL
# =====================================================
def setup_driver_local(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1600,900")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_argument("--log-level=3")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(25)
    return driver


# =====================================================
#  KODE KECAMATAN
# =====================================================
def get_kode_kecamatan_from_json(nama_kecamatan, json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        nama_wilayah = next(iter(data))
        return data[nama_wilayah]["kecamatan"].get(nama_kecamatan)
    except Exception as e:
        print("JSON error:", e)
        return None


# =====================================================
#  OPTIMIZED REQUEST SESSION (keep-alive pool)
# =====================================================
def create_fast_session():
    s = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_connections=50, pool_maxsize=50, max_retries=2)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/121 Safari/537.36"
    })
    return s


# =====================================================
#  FETCH TAB-4 (KONTAK)
# =====================================================
def fetch_contact(url, selected_fields, session):
    result = {}
    try:
        resp = session.get(url, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        if any(f in selected_fields for f in ["Telepon", "Email", "Website"]):
            for row in soup.select("table tr"):
                tds = row.find_all("td")
                if len(tds) >= 4:
                    label = tds[1].get_text(strip=True).lower()
                    val = tds[3].get_text(strip=True)
                    if "telepon" in label:
                        result["Telepon"] = val if len(val) > 4 else "-"
                    elif "email" in label:
                        result["Email"] = val
                    elif "website" in label:
                        result["Website"] = "-" if val in ["http://-", "https://-"] else val
    except Exception:
        pass
    return result


# =====================================================
#  FETCH PROFIL SEKOLAH (Tab-1 redirect)
# =====================================================
def fetch_school_profile(url_tabs, session):
    result = {
        "Alamat": "-", "Kepala Sekolah": "-",
        "Jumlah Siswa Laki-laki": "-", "Jumlah Siswa Perempuan": "-"
    }
    try:
        resp_ref = session.get(url_tabs, timeout=10)
        soup_ref = BeautifulSoup(resp_ref.text, "html.parser")
        link_tag = soup_ref.find("a", href=lambda x: x and "sekolah.data" in x)
        if not link_tag:
            return result

        sekolah_url = link_tag["href"].strip()
        resp = session.get(sekolah_url, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")

        #  Alamat
        alamat = soup.select_one("font.small")
        if alamat:
            result["Alamat"] = alamat.text.replace("(master referensi)", "").strip()

        #  Kepala Sekolah
        for li in soup.select("li.list-group-item"):
            txt = li.get_text(strip=True)
            if "Kepala Sekolah" in txt:
                result["Kepala Sekolah"] = txt.split(":", 1)[-1].strip()
                break

        #  Jumlah Siswa
        div_stat = soup.find("div", class_="col-xs-12 col-md-3 text-left")
        if div_stat:
            tag_m = div_stat.find(string=lambda t: "Siswa Laki-laki" in t)
            if tag_m:
                font_m = tag_m.find_next("font", class_="text-info")
                if font_m:
                    result["Jumlah Siswa Laki-laki"] = font_m.text.strip()

            tag_f = div_stat.find(string=lambda t: "Siswa Perempuan" in t)
            if tag_f:
                font_f = tag_f.find_next("font", class_="text-info")
                if font_f:
                    result["Jumlah Siswa Perempuan"] = font_f.text.strip()

    except Exception as e:
        print(f"Gagal ambil profil sekolah: {e}")
    return result


# =====================================================
#  MAIN SCRAPER
# =====================================================
def get_sd_mi_schools_fast_local(kode_kecamatan, nama_kecamatan, selected_fields, progress=gr.Progress()):
    driver = setup_driver_local(True)
    session = create_fast_session()
    sekolah_list, urls = [], []

    need_detail = any(f in selected_fields for f in
                      ["Alamat", "Kepala Sekolah", "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan",
                       "Telepon", "Email", "Website"])

    for jenjang, value in [("SD", "5"), ("MI", "9")]:
        url = f"https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3/all/{value}/all"
        print(f"{jenjang} - {nama_kecamatan}")
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr")))
        try:
            Select(driver.find_element(By.NAME, "table1_length")).select_by_value("100")
            time.sleep(0.5)
        except:
            pass

        for r in driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr"):
            data = {}
            if "Nama Sekolah" in selected_fields:
                data["Nama Sekolah"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
            if "NPSN" in selected_fields:
                data["NPSN"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
            if "Status" in selected_fields:
                data["Status"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text.strip()
            if "Kelurahan" in selected_fields:
                data["Kelurahan"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text.strip()
            if need_detail:
                link = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                urls.append((link, data))
            else:
                sekolah_list.append(data)

    driver.quit()

    if need_detail:
        print(f"Fetch {len(urls)} sekolah (parallel)...")
        with ThreadPoolExecutor(max_workers=25) as executor:
            futures = {
                executor.submit(
                    lambda l, base: {
                        **base,
                        **fetch_contact(l, selected_fields, session),
                        **fetch_school_profile(l, session)
                    }, link, base
                ): (link, base) for link, base in urls
            }
            for i, f in enumerate(as_completed(futures)):
                sekolah_list.append(f.result())
                if (i + 1) % 10 == 0:
                    progress((i + 1) / len(futures))

    #  Sorting berdasar kolom pertama yang dipilih
    if selected_fields:
        sort_key = selected_fields[0]
        sekolah_list.sort(key=lambda x: str(x.get(sort_key, "")).lower())

    print(f"{len(sekolah_list)} sekolah {nama_kecamatan}")
    return sekolah_list


# =====================================================
#  SAVE CSV
# =====================================================
def save_school_list_by_kecamatan(nama_kecamatan, selected_fields):
    kode = get_kode_kecamatan_from_json(nama_kecamatan)
    data = get_sd_mi_schools_fast_local(kode, nama_kecamatan, selected_fields)
    if not data:
        return f" Tidak ada data '{nama_kecamatan}'", None

    os.makedirs("output", exist_ok=True)
    path = f"output/list_sd_mi_{nama_kecamatan.lower().replace(' ', '_')}.csv"
    cols = [c for c in [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ] if c in selected_fields]

    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        w.writerows(data)
    return f"{len(data)} sekolah disimpan ke '{path}'", path


# =====================================================
# GRADIO
# =====================================================
def run_scraper(nama_kecamatan, selected_fields):
    if nama_kecamatan in ["", "-- Pilih Kecamatan --"]:
        yield "Pilih kecamatan.", gr.update(visible=False)
        return
    if not selected_fields:
        yield "Pilih minimal 1 kolom.", gr.update(visible=False)
        return

    t0 = time.time()
    status, path = save_school_list_by_kecamatan(nama_kecamatan, selected_fields)
    dur = int(time.time() - t0)
    m, s = divmod(dur, 60)
    time_str = f"\n{m} menit {s} detik" if m else f"\n{s} detik"

    yield f"{status}{time_str}", gr.update(value=path, visible=True if path else False)


def create_gradio_ui(json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    nama_wilayah = next(iter(data))
    kec_list = sorted(data[nama_wilayah]["kecamatan"].keys())

    fields = ["Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
              "Alamat", "Telepon", "Email", "Website",
              "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"]

    with gr.Blocks(title="Scraper SD/MI Semarang — v2.0") as demo:
        gr.Markdown("## Scraper SD/MI Kab. Semarang — v2.0 Hybrid (Selenium + Requests + Thread Pool Executor)")
        kec = gr.Dropdown(label="Pilih Kecamatan",
                          choices=["-- Pilih Kecamatan --"] + kec_list,
                          value="-- Pilih Kecamatan --")
        kolom = gr.CheckboxGroup(label="Kolom CSV", choices=fields)
        btn = gr.Button("Mulai Scrape")
        stat = gr.Textbox(label="Status", lines=3)
        file = gr.File(label="File CSV", visible=False)
        btn.click(fn=run_scraper, inputs=[kec, kolom], outputs=[stat, file])
    return demo


if __name__ == "__main__":
    ui = create_gradio_ui()
    ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


SD - Ambarawa
MI - Ambarawa
Fetch 34 sekolah (parallel)...
34 sekolah Ambarawa
